In [7]:
# %%time
"""

重投影时，重采样为每一个像素是十米乘十米

重投影，投影过程中可以设置以下信息
gdal.Warp(
    xRes,yRes:两个方向上的分辨率；
    srcNodata：原来数据的Nodata值
    dstNodata：输出数据的Nodata值
    dstSRS：输出的投影坐标系，可以读取影像的：
            Raster = gdal.Open(InputImage, gdal.GA_ReadOnly)
            Projection = Raster.GetProjectionRef()
    resampleAlg:重采样方式,算法包括：
            import gdalconst
            gdalconst.GRA_NearestNeighbour：near
            gdalconst.GRA_Bilinear:bilinear
            gdalconst.GRA_Cubic:cubic
            gdalconst.GRA_CubicSpline:cubicspline
            gdalconst.GRA_Lanczos:lanczos
            gdalconst.GRA_Average:average
            gdalconst.GRA_Mode:mode
    )

"""
import numpy as np
from osgeo import gdal,gdalconst
import os,datetime

# Open datasets，此处的tif文件为模板文件，其提供目标投影坐标
InputImage = r'C:\zyh\ktywork\20250703sichuan_agriculture\muban.tif'
Raster = gdal.Open(InputImage, gdal.GA_ReadOnly)
Projection = Raster.GetProjectionRef()

i=0
# 需要重投影的tif输入文件所在文件夹
inDir = r'C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\\'+str(i)
# 投影后tif输出文件存放文件夹
outDir = r'C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\\'+str(i)

# 如果没有输出文件夹，创建文件夹
if not os.path.exists(outDir):
        os.makedirs(outDir)
inList = [name for name in os.listdir(inDir)]
print(inList)

for file in inList:
    if file.endswith('.tif'):
        input_raster = os.path.join(inDir,file)
    # 输出文件的完整路径
        output_raster = os.path.join(outDir,file)
        if not os.path.exists(output_raster):
            OutTile = gdal.Warp(output_raster, input_raster,
                    dstSRS=Projection,
                    resampleAlg=gdalconst.GRA_NearestNeighbour,
                    dstNodata=0,
                    xRes=10,
                    yRes=10
                    )
        OutTile =None
        print(inList.index(file))
print("END!")

['20240912.tif', '20240912.tif.ovr']
0
END!


In [8]:
# 修改文件名-->日期+0+行数+0+列数
i=0
print(i)
filepath = r'C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\\'+str(i)
ls=os.listdir(filepath)
for file in ls:
    if file.endswith('.tif'):
        oldname=str(os.path.join(filepath,file))
        ds = gdal.Open(oldname)
        cols=ds.RasterXSize
        rows=ds.RasterYSize
        del ds
        sttr=str(file[:8])
#         print(sttr)
        newname=filepath+"\\"+sttr+"0"+str(cols)+"0"+str(rows)+".tif"
#         print(oldname)
        print(newname)
    
        if len(os.path.basename(newname).strip(".tif"))==len('20240104011110118'):
            os.rename(oldname,newname)

0
C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\\0\20240210013960924.tif
C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\\0\20240311012330924.tif
C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\\0\20240525012330924.tif
C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\\0\20240813012330924.tif
C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\\0\20240823012330924.tif
C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\\0\20240828012330924.tif
C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\\0\20240902012330924.tif
C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\\0\20240912012330924.tif


FileExistsError: [WinError 183] 当文件已存在时，无法创建该文件。: 'C:\\zyh\\ktywork\\20250703sichuan_agriculture\\sichuan\\time_series\\luorun\\cty\\\\0\\20240912.tif' -> 'C:\\zyh\\ktywork\\20250703sichuan_agriculture\\sichuan\\time_series\\luorun\\cty\\\\0\\20240912012330924.tif'

In [3]:
# 样本矢量转栅格

import sys,os
import geopandas as gpd
import rasterio
from osgeo import gdal
from rasterio import features
from rasterio.crs import CRS
from rasterio.transform import Affine

class Polygon2Raster:
    def polygon_to_raster(self,shp,raster,pixel,field,field1,code=4326):
        '''
        矢量转栅格
        :param shp: 输入矢量全路径，字符串，无默认值
        :param raster: 输出栅格全路径，字符串，无默认值
        :param pixel: 像元大小，与矢量坐标系相关
        :param field: 栅格像元值字段
        :param Code: 输出坐标系代码，默认为4326
        :return: None
        '''

        # 判断字段是否存在
        shapefile = gpd.read_file(shp)
        if not field in shapefile.columns:
            raise Exception ('输出字段不存在')
        # 判断数据类型
        f_type = shapefile.dtypes.get(field)
        if 'int' in str(f_type):
            shapefile[field] = shapefile[field].astype('int16')
            dtype = 'int16'
        elif 'float' in str(f_type):
            shapefile[field] = shapefile[field].astype('float32')
            dtype = 'float32'
        else:
            raise Exception ('输入字段数据类型为{}，无法进行栅格化操作'.format(f_type))

        bound = shapefile.bounds
        print(bound)

        width = int((bound.get('maxx').max()-bound.get('minx').min())/pixel)
        height = int((bound.get('maxy').max()-bound.get('miny').min())/pixel)
        transform = Affine(pixel, 0.0, bound.get('minx').min(),
               0.0, -pixel, bound.get('maxy').max())
        InputImage = r"C:\zyh\data\fishnet\muban.tif"
        Raster = gdal.Open(InputImage, gdal.GA_ReadOnly)
        Projection = Raster.GetProjectionRef()
        

        meta = {'driver': 'GTiff',
                'dtype': dtype,
                'nodata': 0,
                'width': width,
                'height': height,
                'count': 2,
                'crs': Raster.GetProjectionRef(),
                'transform': transform}
        print(meta)

        with rasterio.open(raster, 'w+', **meta) as out:
            out_arr = out.read(1)
            shapes = ((geom,value) for geom, value in zip(shapefile.get('geometry'), shapefile.get(field)))
            burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
            out.write_band(1, burned)
            out_arr2 = out.read(2)
            shapes1 = ((geom,value) for geom, value in zip(shapefile.get('geometry'), shapefile.get(field1)))
            burned2 = features.rasterize(shapes=shapes1, fill=0, out=out_arr2, transform=out.transform)
            out.write_band(2, burned2)

if __name__ == '__main__':
    
    # 样本路径  shp需要设置投影 以米为单位
    # filepath=r"C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\sample\luorun\\"+str(i) 
    filepath=r"C:\Users\aircas\Desktop\sample"
    ls=os.listdir(filepath)
    for file in ls:
        if file.endswith('.shp'):
            in_shp=str(os.path.join(filepath,file))
            out_tif = r'C:\Users\aircas\Desktop\sample/'+os.path.basename(in_shp).strip('.shp')+'.tif'
            pixel_size = 10
            field = 'typecode'
            field1= 'num'
            Polygon2Raster().polygon_to_raster(in_shp,out_tif,pixel_size,field,field1)
            print(file+'finish')

              minx          miny           maxx          maxy
0    406537.337928  4.112299e+06  406617.463668  4.112383e+06
1    406618.636656  4.112977e+06  406650.511666  4.113028e+06
2    406662.216624  4.113491e+06  406708.486247  4.113547e+06
3    406583.179133  4.113628e+06  406626.339710  4.113663e+06
4    406249.491365  4.113709e+06  406300.417927  4.113775e+06
..             ...           ...            ...           ...
746  388690.134778  4.140065e+06  388797.516447  4.140349e+06
747  392396.646273  4.142679e+06  392544.596670  4.142855e+06
748  383152.205320  4.143090e+06  383398.023306  4.143329e+06
749  382911.872874  4.143182e+06  383209.634163  4.143453e+06
750  377885.797342  4.146071e+06  377907.731850  4.146101e+06

[751 rows x 4 columns]
{'driver': 'GTiff', 'dtype': 'int16', 'nodata': 0, 'width': 6191, 'height': 6597, 'count': 2, 'crs': 'PROJCS["WGS 84 / UTM zone 44N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],

In [10]:
# 样本栅格扩充：把样本栅格范围转成和影像大小一样

from osgeo import gdal
import os

# filepath=r'C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\\'+str(i)
filepath=r'C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\0'
rasterfile=str(os.path.join(filepath,os.listdir(filepath)[2]))
# 注意这一步需要文件夹中全部都为tif格式文件，不要有enp等
print(rasterfile)

# ybfile=r'D:\xinjiang\hetian\sample\ht_sample\bu\14/'+str(i)+'.tif'
# out_file=r'D:\xinjiang\hetian\sample\ht_sample\bu\14/'+str(i)+'kc.tif'
ybfile=r"C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\sample\luorun\luorun_sample_prj.tif"
out_file=r"C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\sample\luorun\luorun_sample_prjkc.tif"
ds=gdal.Open(rasterfile)
ds2=gdal.Open(ybfile)

if ds2 is None:
    print("打开失败！")
else:
    int_data=ds.GetRasterBand(1)
    yb_data = ds2.GetRasterBand(1).ReadAsArray()
    yb_data1 = ds2.GetRasterBand(2).ReadAsArray()
#     yb_data[yb_data <0]=0
#     yb_data[yb_data==np.nan]=0
    gtif_driver=gdal.GetDriverByName("GTiff")
    out_ds=gtif_driver.Create(out_file,int_data.XSize,int_data.YSize,2,int_data.DataType)
    in_datatf=ds.GetGeoTransform()
    print("muban",in_datatf)
    out_ds.SetProjection(ds.GetProjection())
    out_ds.SetGeoTransform(in_datatf)    
    out_band=out_ds.GetRasterBand(1)
    out_band1=out_ds.GetRasterBand(2)
#     zeros=np.zeros(int_data.XSize,int_data.YSize)
#     out_band.WriteArray(zeros)
    yb_tf=ds2.GetGeoTransform()
    print("yb",yb_tf)
    yb_lx=yb_tf[0]
    yb_ly=yb_tf[3]
    l_x=int(-(in_datatf[0]-yb_lx)/in_datatf[1])
    l_y=int((in_datatf[3]-yb_ly)/-yb_tf[5])
    print(l_x,l_y)
    out_band.WriteArray(yb_data,l_x,l_y)
    out_band1.WriteArray(yb_data1,l_x,l_y)
    print(out_band.XSize,out_band.YSize)
#     out_band[out_band <0]=0
#     out_band[out_data==np.nan]=0

    out_ds.FlushCache()
    del out_ds
    del ds
    del ds2
    print("End!")


C:\zyh\ktywork\20250703sichuan_agriculture\sichuan\time_series\luorun\cty\0\20240813012330924.tif
muban (446009.4169853241, 10.0, 0.0, 3139563.471365611, 0.0, -10.0)
yb (451664.06960917736, 10.0, 0.0, 3135927.440166229, 0.0, -10.0)
565 363
1233 924
End!


In [88]:
# 批量重投影
import numpy as np
from osgeo import gdal,gdalconst
import os,datetime

# Open datasets，此处的tif文件为模板文件，其提供目标投影坐标
InputImage = r'C:\zyh\data\fishnet\muban.tif'
Raster = gdal.Open(InputImage, gdal.GA_ReadOnly)
Projection = Raster.GetProjectionRef()

# 定义需要处理的i的值列表
index = [62,63,64,67]

for i in index:
    # 需要重投影的tif输入文件所在文件夹
    inDir = r'C:\zyh\data\fishnet\\' + str(i)
    # 投影后tif输出文件存放文件夹
    outDir = r'C:\zyh\data\fishnet_cty\\' + str(i)

    # 如果没有输出文件夹，创建文件夹
    if not os.path.exists(outDir):
        os.makedirs(outDir)

    # 获取输入文件夹中的所有文件名
    inList = [name for name in os.listdir(inDir)]
    print(f"Processing folder {i}: {inList}")

    for file in inList:
        if file.endswith('.tif'):
            # 输入文件的完整路径
            input_raster = os.path.join(inDir, file)
            # 输出文件的完整路径
            output_raster = os.path.join(outDir, file)
            if not os.path.exists(output_raster):
                # 调用gdal.Warp进行重投影
                OutTile = gdal.Warp(output_raster, input_raster,
                                     dstSRS=Projection,
                                     resampleAlg=gdalconst.GRA_NearestNeighbour,
                                     dstNodata=0,
                                     xRes=10,
                                     yRes=10)
                OutTile = None
            # print(f"Processed file {file} in folder {i}")
    print(f"Finished processing folder {i}")

print("END!")

Processing folder 62: ['20240104T053221_20240104T053220.tif', '20240119T053149_20240119T053143.tif', '20240124T053111_20240124T053108.tif', '20240228T052749_20240228T053352.tif', '20240309T052639_20240309T053132.tif', '20240324T052651_20240324T053033.tif', '20240403T052641_20240403T053120.tif', '20240408T052649_20240408T053621.tif', '20240423T052651_20240423T053016.tif', '20240503T052651_20240503T053613.tif', '20240508T052649_20240508T053113.tif', '20240518T052649_20240518T052846.tif', '20240607T052649_20240607T053629.tif', '20240612T052651_20240612T053339.tif', '20240707T052649_20240707T053238.tif', '20240717T052649_20240717T053111.tif', '20240727T052649_20240727T053114.tif', '20240826T052649_20240826T053725.tif', '20240905T052649_20240905T052647.tif', '20240915T052649_20240915T053741.tif', '20240925T052649_20240925T053331.tif', '20241005T052649_20241005T053409.tif', '20241010T052731_20241010T053314.tif', '20241015T052719_20241015T052955.tif', '20241030T052941_20241030T053441.tif', '2

In [4]:
# 批量重命名
import os
from osgeo import gdal

index = [0,1,3,14,22,27,29,31,38,44,47,48,50,53,57,58,59,60,62,63,64,67]
for i in index:
    print(i)
    filepath = r'C:\zyh\data\fishnet_cty\\'+str(i)
    ls=os.listdir(filepath)
    for file in ls:
        if file.endswith('.tif'):
            oldname=str(os.path.join(filepath,file))
            ds = gdal.Open(oldname)
            cols=ds.RasterXSize
            rows=ds.RasterYSize
            del ds
            sttr=str(file[:8])
    #         print(sttr)
            newname=filepath+"\\"+sttr+"0"+str(cols)+"0"+str(rows)+".tif"
    #         print(oldname)
            print(newname)
    
            if len(os.path.basename(newname).strip(".tif"))==len('202401040111001182'):
                os.rename(oldname,newname)

0
C:\zyh\data\fishnet_cty\\0\202401040111201186.tif
C:\zyh\data\fishnet_cty\\0\202401090111201186.tif
C:\zyh\data\fishnet_cty\\0\202401190111201186.tif
C:\zyh\data\fishnet_cty\\0\202401240111201186.tif
C:\zyh\data\fishnet_cty\\0\202402130111201186.tif
C:\zyh\data\fishnet_cty\\0\202402280111201186.tif
C:\zyh\data\fishnet_cty\\0\202403090111201186.tif
C:\zyh\data\fishnet_cty\\0\202403190111201186.tif
C:\zyh\data\fishnet_cty\\0\202403240111201186.tif
C:\zyh\data\fishnet_cty\\0\202404080111201186.tif
C:\zyh\data\fishnet_cty\\0\202405030111201186.tif
C:\zyh\data\fishnet_cty\\0\202405080111201186.tif
C:\zyh\data\fishnet_cty\\0\202405180111201186.tif
C:\zyh\data\fishnet_cty\\0\202406120111201186.tif
C:\zyh\data\fishnet_cty\\0\202406170111201186.tif
C:\zyh\data\fishnet_cty\\0\202407270111201186.tif
C:\zyh\data\fishnet_cty\\0\202409050111201186.tif
C:\zyh\data\fishnet_cty\\0\202409250111201186.tif
C:\zyh\data\fishnet_cty\\0\202410150111201186.tif
C:\zyh\data\fishnet_cty\\0\202410200111201186.ti